In [26]:
%matplotlib inline
import numpy as np
import scipy as sp
from itertools import product
import json 
import csv
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [27]:
dftouse=pd.read_csv("dftouse.csv")

In [28]:
del dftouse['Unnamed: 0']
del dftouse['ReserveMet']
del dftouse['MinimumToBid']
dftouse = dftouse[pd.isnull(dftouse.HitCount) == False]

In [29]:
dftouse.head()

,country,itemId,paymentMethod,postalCode,sellingStatus,topRatedListing,AutoPay,AvailableForPickupDropOff,BestOfferEnabled,BidCount,BuyItNowPrice,FinalPrice,GlobalShipping,HandlingTime,HitCount,ListingStatus,ListingType,Location,ProductID,Quantity,QuantityThreshold,ShipToLocations,Title,UserID,FeedbackScore,PositiveFeedbackPercent,SellerBusinessType,NewUser,FeedbackPrivate,nPhotos,conditionId,bestOfferEnabled,buyItNowAvailable,expeditedShipping,shippingType,oneDayShippingAvailable,NegFeedback30Day,NegFeedback365Day,PosFeedback30Day,PosFeedback365Day,auctionLength
0,0,131642735687,0,10590,0,False,False,0,False,41,0,202.09,False,2,190,Completed,0,"South Salem, New York",NaN,1,0,0,iPhone 5s-32gb-Space Gray-unlocked,jdwein1980,19,100.0,1,False,False,3,5,False,False,True,1,False,0,0,5,5,168
1,0,262126933108,0,35004,0,False,False,0,False,4,0,260.00,False,3,32,Completed,0,"Moody, Alabama",NaN,1,0,0,apple iphone 5s 32gb,forwhlrcr1965,565,100.0,1,False,False,2,5,False,False,True,0,False,0,0,3,32,120
2,0,252155964081,0,98104,0,False,False,0,False,3,0,152.50,False,2,168,Completed,0,"Seattle, Washington",168534287,1,0,1,Apple iPhone 5s - 32GB SPACE GRAY AT&T,15-alice,15,94.1,1,False,False,2,5,False,False,False,1,False,0,1,2,17,120
3,0,252163774777,0,98104,0,False,False,0,False,1,0,99.00,False,2,143,Completed,0,"Seattle, Washington",168553370,1,0,0,Apple iPhone 5s - 32GB - Silver (Verizon) Smar...,15-alice,15,94.1,1,False,False,7,9,False,False,True,0,False,0,1,2,17,120
4,0,252171170160,0,98104,0,False,True,0,False,5,0,96.00,False,2,357,Completed,0,"Seattle, Washington",168534287,1,0,0,Apple iPhone 5s - 32GB SPACE GRAY AT&T,15-alice,15,94.1,1,False,False,2,5,False,False,False,1,False,0,1,2,17,120


In [44]:
ZERO_IMPORTANT_VARIABLES = ['BidCount', 'HitCount', 'nPhotos', 'FeedbackScore', 'PositiveFeedbackPercent_x',
                            'conditionId']
STANDARDIZABLE = ['FinalPrice','auctionLength', 'HandlingTime', 'FeedbackScore', 
                  'PositiveFeedbackPercent', 'Quantity']

In [45]:
for v in dftouse.columns:
    l=dftouse[v].unique()
    if len(l) <= 10:
        print v, l

country [0 1]
paymentMethod [0 1]
sellingStatus [0 1]
topRatedListing [False True]
AutoPay [False True]
AvailableForPickupDropOff [0 1]
BestOfferEnabled [False True]
BuyItNowPrice [0 1]
GlobalShipping [False True]
ListingStatus ['Completed' 'Active']
ListingType [0 1]
QuantityThreshold [0 1]
ShipToLocations [0 1]
SellerBusinessType [1 0]
NewUser [False True]
FeedbackPrivate [False True]
conditionId [5 9 0 1 3 4]
bestOfferEnabled [False True]
buyItNowAvailable [False True]
expeditedShipping [True False]
shippingType [1 0]
oneDayShippingAvailable [False True]


In [46]:
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)

In [47]:
mask=np.ones(dftouse.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [48]:
mask

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [49]:
mask.shape, mask.sum()

((9267,), 6486)

In [50]:
from sklearn.preprocessing import StandardScaler
for col in STANDARDIZABLE:
    print col
    valstrain=dftouse[col].values[mask]
    valstest=dftouse[col].values[~mask]
    scaler=StandardScaler().fit(valstrain)
    outtrain=scaler.transform(valstrain)
    outtest=scaler.fit_transform(valstest)
    out=np.empty(mask.shape[0])
    out[mask]=outtrain
    out[~mask]=outtest
    dftouse[col]=out

FinalPrice
auctionLength
HandlingTime
FeedbackScore
PositiveFeedbackPercent
Quantity


In [52]:
lcols=list(dftouse.columns)
lcols.remove(u'sellingStatus')
print len(lcols)

40


In [53]:
ccols=[]
for c in lcols:
    if c not in INDICATORS:
        ccols.append(c)
print len(ccols), len(INDICATORS)
ccols

NameError: name 'INDICATORS' is not defined